In [4]:
import time
import numpy as np
import pandas as pd
import os

from hydesign.assembly.hpp_assembly import hpp_model
from hydesign.examples import examples_filepath

In [6]:
sites = pd.read_csv('overview.csv', index_col=0)
example = 0

ex_site = sites.iloc[int(example),:]

print('Selected example site:')
print('---------------------------------------------------')
print(ex_site.T)

longitude = ex_site['longitude']
latitude = ex_site['latitude']
altitude = ex_site['altitude']
input_ts_fn = os.path.join(os.getcwd(), ex_site['input_ts_fn'])
sim_pars_fn = os.path.join(os.getcwd(), ex_site['sim_pars_fn'])

hpp = hpp_model(
        latitude=latitude,
        longitude=longitude,
        altitude=altitude,
        num_batteries = 10,
        sim_pars_fn = sim_pars_fn,
        input_ts_fn = input_ts_fn,
)

inputs = dict(
clearance =	60,
sp =	287.6391934,
p_rated =	10,
Nwt	=	25,
wind_MW_per_km2 =	4.782274215,
solar_MW =	223.2191223,
surface_tilt =	50,
surface_azimuth = 210,
DC_AC_ratio	=1.295598428,
b_P =75.27402422,
b_E_h=	4.451612903,
cost_of_battery_P_fluct_in_peak_price_ratio	=0.580645161,)

Selected example site:
---------------------------------------------------
case                                              Europe
name                                    France_good_wind
longitude                                      -0.864258
latitude                                       48.744116
altitude                                             302
input_ts_fn      hyd_input/input_ts_France_good_wind.csv
sim_pars_fn                       hyd_input/hpp_pars.yml
price_fn                     hyd_input/2030-EL_PRICE.csv
price_col                                           FR_R
H2_demand_col                    hyd_input/H2_demand.csv
Name: 0, dtype: object

Fixed parameters on the site
-------------------------------
longitude = -0.864258
latitude = 48.744116
altitude = 302

input_ts_fn length is not a complete number of years (hyDesign handles years as 365 days).
The file has been modified and stored in ./input_ts_modified.csv


In [7]:
start = time.time()
outs = hpp.evaluate(**inputs)
hpp.print_design([v for k,v in inputs.items()], outs)
end = time.time()
print('exec. time [min]:', (end - start)/60 )
print(hpp.prob['NPV_over_CAPEX'])


Design:
---------------
clearance [m]: 60.000
sp [W/m2]: 287.639
p_rated [MW]: 10.000
Nwt: 25.000
wind_MW_per_km2 [MW/km2]: 4.782
solar_MW [MW]: 223.219
surface_tilt [deg]: 50.000
surface_azimuth [deg]: 210.000
DC_AC_ratio: 1.296
b_P [MW]: 75.274
b_E_h [h]: 4.452
cost_of_battery_P_fluct_in_peak_price_ratio: 0.581


NPV_over_CAPEX: 0.133
NPV [MEuro]: 57.149
IRR: 0.072
LCOE [Euro/MWh]: 33.649
Revenues [MEuro]: 48.052
CAPEX [MEuro]: 429.258
OPEX [MEuro]: 6.900
Wind CAPEX [MEuro]: 275.586
Wind OPEX [MEuro]: 5.599
PV CAPEX [MEuro]: 65.197
PV OPEX [MEuro]: 1.301
Batt CAPEX [MEuro]: 21.810
Batt OPEX [MEuro]: 0.000
Shared CAPEX [MEuro]: 66.665
Shared OPEX [MEuro]: 0.000
penalty lifetime [MEuro]: 0.000
Mean Annual Electricity Sold [GWh]: 1202.856
GUF: 0.458
grid [MW]: 300.000
wind [MW]: 250.000
solar [MW]: 223.219
Battery Energy [MWh]: 335.091
Battery Power [MW]: 75.274
Total curtailment [GWh]: 56.301
Total curtailment with deg [GWh]: 124.901
Awpp [km2]: 52.276
Apvp [km2]: 2.737
Plant area [km

In [8]:
from hopp.simulation import HoppInterface

c:\Sandbox\Repo\IEAWindTask50\tool-benchmarking\Europe\log\hybrid_systems_2025-12-10T19.39.08.250995.log


c:\Users\mikf\Anaconda3\envs\hydesign4\Lib\site-packages\fastkml\__init__.py:28: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound


In [9]:
hi = HoppInterface(os.path.join(os.getcwd(), "hopp_input/08-wind-solar-france.yaml"))

FLORIS is the system model...


KeyError: 'power_thrust_data_file'

In [ ]:
hi.system.wind._financial_model.FinancialParameters.real_discount_rate = 6
hi.system.pv._financial_model.FinancialParameters.real_discount_rate = 6
hi.system.battery._financial_model.FinancialParameters.real_discount_rate = 6
hi.system.grid._financial_model.FinancialParameters.real_discount_rate = 6

hi.system.wind._financial_model.FinancialParameters.inflation_rate = 2
hi.system.pv._financial_model.FinancialParameters.inflation_rate = 2
hi.system.battery._financial_model.FinancialParameters.inflation_rate = 2
hi.system.grid._financial_model.FinancialParameters.inflation_rate = 2

# Federal tax rate 21%, set state tax rate to 4% (total 25% tax rate)
hi.system.wind._financial_model.FinancialParameters.state_tax_rate = [4]*25
hi.system.pv._financial_model.FinancialParameters.state_tax_rate = [4]*25
hi.system.battery._financial_model.FinancialParameters.state_tax_rate = [4]*25
hi.system.grid._financial_model.FinancialParameters.state_tax_rate = [4]*25

In [ ]:
hi.simulate(25)

In [ ]:
hybrid_plant = hi.system

annual_energies = hybrid_plant.annual_energies
npvs = hybrid_plant.net_present_values
cf = hybrid_plant.capacity_factors

wind_installed_cost = hybrid_plant.wind.total_installed_cost
solar_installed_cost = hybrid_plant.pv.total_installed_cost
battery_installed_cost = hybrid_plant.battery.total_installed_cost
hybrid_installed_cost = hybrid_plant.grid.total_installed_cost

print("Wind Installed Cost [M$]: {}".format(wind_installed_cost/1e6))
print("Solar Installed Cost [M$]: {}".format(solar_installed_cost/1e6))
print("Battery Installed Cost [M$]: {}".format(battery_installed_cost/1e6))
print("Hybrid Installed Cost [M$]: {}\n".format(hybrid_installed_cost/1e6))

print("Wind NPV [M$]: {}".format(hybrid_plant.net_present_values.wind/1e6))
print("Solar NPV [M$]: {}".format(hybrid_plant.net_present_values.pv/1e6))
print("Hybrid NPV [M$]: {}\n".format(hybrid_plant.net_present_values.hybrid/1e6))

print("Annual Energies [kW]",annual_energies)
print("Capacity Factors",cf)
print("NPV [$]", npvs)

print("LCOE [cents/kWh]", hybrid_plant.lcoe_real,"\n")

print("Hybrid LCOE [$/MWh]", hybrid_plant.lcoe_real['hybrid']*10)

In [ ]:
import json

with open("singleowner.json", 'w') as f:
    dat = hybrid_plant.grid._financial_model.export()
    d = dict()
    for k, v in dat.items():
        d.update(v)
    json.dump(d, f)

In [ ]:
euro_per_dollar = 1.0
df = pd.DataFrame({'HOPP': [wind_installed_cost/1e6*euro_per_dollar,
                            solar_installed_cost/1e6*euro_per_dollar,
                            battery_installed_cost/1e6*euro_per_dollar,
                            hybrid_installed_cost/1e6*euro_per_dollar,
                            hybrid_plant.net_present_values.hybrid/1e6*euro_per_dollar,
                           ],
                   'HyDesign': [float(hpp.prob['wpp_cost.CAPEX_w']/1e6),
                                float(hpp.prob['pvp_cost.CAPEX_s']/1e6),
                                float(hpp.prob['battery_cost.CAPEX_b']/1e6),
                                float(hpp.prob['CAPEX']/1e6),
                                float(hpp.prob['NPV']/1e6),
                                ] },
                  index=['CAPEX Wind [M€]',
                         'CAPEX Solar [M€]',
                         'CAPEX Battery [M€]',
                         'CAPEX Hybrid [M€]',
                         'NPV Hybrid [M€]',
                        ])

In [ ]:
df

In [ ]:
hpp.prob.model.list_outputs()